In [ ]:
import numpy as np
import webdataset as wds
import pandas as pd
import itertools

In [ ]:
patient_names = ["pat_1307503"]
file_paths = [f"/innovation_cache/merged_eeg/{patient_name}.univariate.tar" for patient_name in patient_names]
# ds = wds.WebDataset(file_paths, shardshuffle=False).decode("l").to_tuple("segment.npy").map_tuple(lambda sample: {"target": sample.tolist(), "start": pd.Period("01-01-2019", freq='D')})
ds = wds.WebDataset(file_paths, shardshuffle=False).decode("l")

In [ ]:
def generate_wds_to_tsdf(ds):
    """Generate webdataset to AutoGluon timeseries dataset."""
    for sample in ds:
        yield {"target": sample["segment.npy"].tolist(),
               "start": pd.Period("01-01-2019", freq=f'{int(1000000/4096)}us')}

In [ ]:
iterable_dataset = list(itertools.islice(generate_wds_to_tsdf(ds), 5))

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
segment = iterable_dataset[0]["target"]
plt.plot(segment)
plt.show()

In [ ]:
from scipy.io.wavfile import write
rate = int(len(segment)/20)
scaled = np.int16((segment / np.max(np.abs(segment))) * 32767)

In [ ]:
write("test.wav", rate, scaled)

In [ ]:
plt.plot(scaled)
plt.show()